In [1]:
from helpers_given import *
from preprocess import *
from helpers import *

In [2]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("dataset/", sub_sample=False)

y_train loaded
x_train loaded
x_test loaded


In [3]:
y_01 = convert_minus1_to_0(y_train)
nb_col = x_train.shape[1]
nb_rows = x_train.shape[0]
print(nb_col)
print(nb_rows)
#get the features names
with open('dataset/x_train.csv', 'r') as f:
    features_string = f.readline()
    features = features_string.split(',')
features = features[1:]

321
328135


In [4]:
features_ids = np.arange(len(features))

In [25]:
nb_nan = nb_of_nans(x_train)
# Get reduced data
reduced_data, reduced_features, removed_features = removing_features(nb_nan, features, x_train)
# remove the same columns from x_test
reduced_x_test = np.delete(x_test, removed_features, 1)

In [8]:
reduced_ids = np.delete(features_ids, removed_features)

In [9]:
print(reduced_data.shape)

(328135, 139)


In [16]:
print(reduced_ids.shape)

(139,)


In [10]:
check_x_1 = x_train[:, reduced_ids]

with the right preprocessing

In [ ]:
# Get feature correlation dictionary
feature_correlation_dict = create_dictionary_from_correlation(reduced_data,reduced_features,0.6)

max_corrr_feature_dict = {}


for key, val in feature_correlation_dict.items():
    max_corrr_feature_dict[key] = len(val)
# Sort the dictionary by value in descending order
max_corrr_feature_dict = {k: v for k, v in sorted(max_corrr_feature_dict.items(), key=lambda item: item[1], reverse=True)}

features_to_drop = []
for key in max_corrr_feature_dict.keys():
    features_to_drop.append(key)
    if len(features_to_drop) == 30:
        break

# Define features to keep
features_to_keep = []
for feature in reduced_features:
    if feature not in features_to_drop:
        features_to_keep.append(feature)

# Also replace some features with their calculated counterparts
origin_calculated_features = {
    'WEIGHT2' : 'WTKG3',
    'HEIGHT3' : 'HTM4',
    'ALCDAY5' : '_DRNKWEK',
    'FRUITJU1' : 'FTJUDA1_',
    'FRUIT1' : 'FRUTDA1_',
    'FVBEANS' : 'BEANDAY_',
    'FVGREEN' : 'GRENDAY_',
    'FVORANG' : 'ORNGDAY_',
    'VEGETAB1' : 'VEGEDA1_',
    'STRENGTH' : 'STRFREQ_'
}

# In features_to_keep replace the key of origin_calculated_features with the value
for key, val in origin_calculated_features.items():
    for i, feature in enumerate(features_to_keep):
        if key == feature:
            features_to_keep[i] = val

# Drop duplicates
features_to_keep = list(set(features_to_keep))

In [26]:


# Get the indices of the selected features
selected_features_indices = []
selected_features_indices_original = []
for feature in features_to_keep:
    selected_features_indices.append(reduced_features.index(feature))
    selected_features_indices_original.append(features.index(feature))

selected_features_indices = sorted(selected_features_indices)
selected_features_indices_original = sorted(selected_features_indices_original)

# Create a new dataset with keeping the features that are in the selected_features_indices
reduced_data = reduced_data[:,selected_features_indices]
reduced_x_test = reduced_x_test[:,selected_features_indices]

# Also remove the features from the reduced_features list
reduced_features_2 = []
for feature in reduced_features:
    if feature in features_to_keep:
        reduced_features_2.append(feature)

In [29]:

# Remove redundant features
redundant_features = [ 'FMONTH','IDATE','IMONTH','IDAY','IYEAR', 'SEQNO', '_STATE', '_PSU', ]
# Get the indices of these features
redundant_features_indices = []
redundant_features_indices_original = []
for feature in redundant_features:
    redundant_features_indices.append(reduced_features_2.index(feature))
    redundant_features_indices_original.append(features.index(feature))

# Create a new dataset with removing the features that are in the selected_features_indices
reduced_data = np.delete(reduced_data, redundant_features_indices, 1)
reduced_features_2 = [reduced_features_2[i] for i in range(len(reduced_features_2)) if i not in redundant_features_indices]
reduced_x_test = np.delete(reduced_x_test, redundant_features_indices, 1)

# Replace nine values with NaNs
replace_nine_with_nan(reduced_data)
replace_nine_with_nan(reduced_x_test)

# Replace seven values with NaNs
replace_seven_with_nan(reduced_data)
replace_seven_with_nan(reduced_x_test)

# Replace 99 values with NaNs
replace_99_with_nan(reduced_data)
replace_99_with_nan(reduced_x_test)

# For the _DRNKWEK feature, replace 9990 with NaN
for i in range(reduced_data.shape[0]):
    if reduced_data[i, reduced_features_2.index('_DRNKWEK')] == 9990:
        reduced_data[i, reduced_features_2.index('_DRNKWEK')] = np.nan
for i in range(reduced_x_test.shape[0]):
    if reduced_x_test[i, reduced_features_2.index('_DRNKWEK')] == 9990:
        reduced_x_test[i, reduced_features_2.index('_DRNKWEK')] = np.nan

In [30]:
reduced_ids_2 = [i for i in selected_features_indices_original if i not in redundant_features_indices_original]

In [31]:
print(reduced_data.shape[1])
print(len(reduced_ids_2))

92
92


In [32]:
check_x = x_train[:, reduced_ids_2]

In [34]:
check_x, reduced_data

(array([[ 2.,  1.,  5., ...,  3.,  4.,  2.],
        [ 4., 88., 88., ...,  3.,  4., nan],
        [ 2., 77., 77., ...,  2.,  3.,  2.],
        ...,
        [ 3., 88.,  1., ...,  3.,  4.,  2.],
        [ 3., 88., 88., ...,  3.,  4.,  2.],
        [ 2.,  7.,  7., ...,  2.,  3.,  2.]]),
 array([[ 2.,  1.,  5., ...,  3.,  4.,  2.],
        [ 4.,  0.,  0., ...,  3.,  4., nan],
        [ 2., nan, nan, ...,  2.,  3.,  2.],
        ...,
        [ 3.,  0.,  1., ...,  3.,  4.,  2.],
        [ 3.,  0.,  0., ...,  3.,  4.,  2.],
        [ 2.,  7.,  7., ...,  2.,  3.,  2.]]))

In [28]:
replace_nine_with_nan(check_x)
replace_seven_with_nan(check_x)
replace_99_with_nan(check_x)

In [29]:
for i in range(check_x.shape[0]):
    if check_x[i, reduced_features_2.index('_DRNKWEK')] == 9990:
        check_x[i, reduced_features_2.index('_DRNKWEK')] = np.nan

In [32]:
print(np.sum(check_x!=reduced_data))

1098153


In [ ]:
# Remove outliers
reduced_data = clean_outliers_modified(reduced_data)
reduced_test = clean_outliers_modified(reduced_x_test)

# Replace NaNs with medians
reduced_median = replace_NaN(reduced_data, method='median')
reduced_median_test = replace_NaN(reduced_test, method='median')

# Standardize the data
standardized_x = standardize_data(reduced_median)
standardized_test = standardize_data(reduced_median_test)

In [ ]:
def replace_nan(tx, method='mean'):
    if method=='mean':
        tx = np.where(np.isnan(tx), np.ma.array(tx, mask=np.isnan(tx)).mean(axis=0), tx) 
    elif method=='median':
        tx = np.where(np.isnan(tx), np.ma.array(tx, mask=np.isnan(tx)).median(axis=0), tx) 
    return tx

In [ ]:
print(np.isnan(reduced_data)[np.isnan(reduced_data)==True].shape)

In [ ]:
reduced_2 = reduced_data
reduced_2[np.isnan(reduced_2)] = np.nanmedian(reduced_2, axis=0)

In [ ]:
# GD with all preprocess
F1 = np.zeros(5)
accuracy = np.zeros(5)
losses = np.zeros(5)
for degree in range(5):
    tx = build_poly(standardized_x, degree)
    w0 = np.zeros(tx.shape[1])
    w_mse_gd, loss_mse_gd = mean_squared_error_gd(y_01, tx, w0, max_iters = 50, gamma = 0.01)
    y_mse_gd = convert_predict(tx @ w_mse_gd)
    F1[degree] = compute_f1(y_01, y_mse_gd)
    accuracy[degree] = compute_accuracy(y_01, y_mse_gd)
    losses[degree] = loss_mse_gd

In [ ]:
print(accuracy)
print(F1)
print(losses)

In [ ]:
col_means = np.nanmean(x_train, axis = 0)
tx = x_train
tx_clean = np.where(np.isnan(tx), np.ma.array(tx, mask=np.isnan(tx)).mean(axis=0), tx) 

In [ ]:
print(tx_clean)

In [ ]:
def prediction(tx, w):
    return convert_predict(tx@w)

In [ ]:
w, loss = mean_squared_error_gd(y_01, tx_clean, np.zeros(tx_clean.shape[1]), max_iters=5, gamma=0.01)
a = compute_accuracy(y_01, prediction(tx_clean, w))

In [ ]:
print(a)

In [ ]:
print(y_01)

In [ ]:
tx = x_train
replace_NaN(tx)

In [ ]:
w, loss = mean_squared_error_gd(y_01, tx_clean, np.zeros(tx_clean.shape[1]), max_iters=50, gamma=0.01)
a = compute_accuracy(y_01, prediction(tx_clean, w))

In [ ]:
print(w)

In [ ]:

k_fold = 4
max_iter = 50
k_indices = build_k_indices(y_01, k_fold, seed=1)
gammas = [0.01, 1e-2, 1e-4, 1e-5]

for gamma_ in gammas:
    accuracy = []
    for k in range(k_fold):
        te_indice = k_indices[k]
        tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
        tr_indice = tr_indice.reshape(-1)
        y_te = y_01[te_indice]
        y_tr = y_01[tr_indice]
        tx_te = tx_clean[te_indice]
        tx_tr = tx_clean[tr_indice]
        w, loss_tr = mean_squared_error_gd(y_tr, tx_tr, np.zeros(tx_tr.shape[1]), max_iter, gamma_)
        a = compute_accuracy(y_te, prediction(tx_te, w))
        accuracy.append(a)
    print("for gamma= ", gamma_)
    print(np.mean(accuracy), np.std(accuracy))